# Estimated time of arrival of Food Delivery

The first we need to have estimation time of arrival for customers. Customers can prepare to recieve Food proactively. If customers have urgent tasks, they can solve problem effectively and proactively.
Beside that this estimation help company manage orders and shippers. At worst, the orders are canceled or late, customer service give a resolution for customers immediately

## Data Processing

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

data = pd.read_csv(R'F:\thanh\da_test.csv')
data.head()

,week_day,order_created_at_time,restaurant_id_h,order_id_h,noised_restaurant_latitude,noised_restaurant_longitude,noised_drop_latitude,noised_drop_longitude,resto_avg_driver_arrive_resto_time,resto_avg_merchant_prepare_time,resto_avg_delivery_time,no_items,restaurant_category_id,actual_time_arrival
0,Wednesday,0:03:01,f5882bd952cdb9e416a2c951d7f16935a755d9bf19fb5b...,92e4edf2207b9fd60fb9972a1bcd62d34d4b8e654f0963...,10.779663,106.604650,10.751649,106.609754,304.428571,520.857143,652.428571,2,15,0:28:13
1,Wednesday,0:03:11,96abafeb1e03c584e9e66ddae730798c4ab63d8bcb8191...,6a2338d70aa87d482e5057e7ccc0646c5e21f12793e863...,10.798146,106.643499,10.808109,106.632191,254.941177,280.000000,420.941176,2,4,0:17:01
2,Wednesday,0:04:12,c36562c53838cb8ed23e9de694b67c8f42ebd246ce5073...,8e700cfddfaf6b79989d7ddcdc91f10a739771a444b0a8...,10.796208,106.695271,10.791213,106.686658,220.390244,474.097561,497.658537,1,9,0:16:55
3,Wednesday,0:04:20,d096e644f8596611ff65c5b07aed9f9d17c2ec3158edc6...,70319476604a22c258c394d9b8c879d6d32d511bd2b402...,10.753824,106.683894,10.760239,106.668003,201.444444,278.907407,583.148148,1,12,0:17:42
4,Wednesday,0:06:57,3f8de1592f374f220e4be4456c7c0ec226729af98e99ec...,77e06752127e561d37876418b65159ec9aaa1ad4f86453...,10.796606,106.704031,10.789168,106.714135,213.000000,549.285714,674.142857,1,17,0:28:33


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111536 entries, 0 to 111535
Data columns (total 14 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   week_day                            111536 non-null  object 
 1   order_created_at_time               111536 non-null  object 
 2   restaurant_id_h                     111536 non-null  object 
 3   order_id_h                          111536 non-null  object 
 4   noised_restaurant_latitude          111536 non-null  float64
 5   noised_restaurant_longitude         111536 non-null  float64
 6   noised_drop_latitude                111536 non-null  float64
 7   noised_drop_longitude               111536 non-null  float64
 8   resto_avg_driver_arrive_resto_time  111536 non-null  float64
 9   resto_avg_merchant_prepare_time     111536 non-null  float64
 10  resto_avg_delivery_time             111536 non-null  float64
 11  no_items                  

In [3]:
# Remove duplicate values
data.drop_duplicates(subset = data.columns.values[:-1], keep = 'first', inplace = True)
#Delete datas which we don't use them
data = data.dropna()
data.drop_duplicates(subset = data.columns.values[:-1], keep = 'first', inplace = True)
#Delete datas which we don't use them
data = data.dropna()

In [4]:
data.isnull().sum()

week_day                              0
order_created_at_time                 0
restaurant_id_h                       0
order_id_h                            0
noised_restaurant_latitude            0
noised_restaurant_longitude           0
noised_drop_latitude                  0
noised_drop_longitude                 0
resto_avg_driver_arrive_resto_time    0
resto_avg_merchant_prepare_time       0
resto_avg_delivery_time               0
no_items                              0
restaurant_category_id                0
actual_time_arrival                   0
dtype: int64

In [5]:
missing_values = data.isnull().sum().sort_values(ascending = False)
percentage = (missing_values/len(data))*100
missing_values = pd.concat([missing_values, percentage], axis = 1, keys= ['Missing values', 'Percentage Missing'])
print(missing_values)

                                    Missing values  Percentage Missing
week_day                                         0                 0.0
order_created_at_time                            0                 0.0
restaurant_id_h                                  0                 0.0
order_id_h                                       0                 0.0
noised_restaurant_latitude                       0                 0.0
noised_restaurant_longitude                      0                 0.0
noised_drop_latitude                             0                 0.0
noised_drop_longitude                            0                 0.0
resto_avg_driver_arrive_resto_time               0                 0.0
resto_avg_merchant_prepare_time                  0                 0.0
resto_avg_delivery_time                          0                 0.0
no_items                                         0                 0.0
restaurant_category_id                           0                 0.0
actual

In [6]:
# Set the earth's radius (in kilometers)
R = 6371

# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  
# Calculate the distance between each pair of points
data['distance'] = np.nan

for i in range(len(data)):
    data.loc[i, 'distance'] = distcalculate(data.loc[i, 'noised_restaurant_latitude'], 
                                        data.loc[i, 'noised_restaurant_longitude'], 
                                        data.loc[i, 'noised_drop_latitude'], 
                                        data.loc[i, 'noised_drop_longitude'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111536 entries, 0 to 111535
Data columns (total 15 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   week_day                            111536 non-null  object 
 1   order_created_at_time               111536 non-null  object 
 2   restaurant_id_h                     111536 non-null  object 
 3   order_id_h                          111536 non-null  object 
 4   noised_restaurant_latitude          111536 non-null  float64
 5   noised_restaurant_longitude         111536 non-null  float64
 6   noised_drop_latitude                111536 non-null  float64
 7   noised_drop_longitude               111536 non-null  float64
 8   resto_avg_driver_arrive_resto_time  111536 non-null  float64
 9   resto_avg_merchant_prepare_time     111536 non-null  float64
 10  resto_avg_delivery_time             111536 non-null  float64
 11  no_items                  

In [8]:
import datetime
data['start_time'] = pd.to_datetime(data['order_created_at_time'], format='%H:%M:%S')
data['end_time'] = pd.to_datetime(data['actual_time_arrival'], format='%H:%M:%S')

In [9]:
df=data[(data['end_time']>data['start_time']) | (data['end_time']==data['start_time'])]

In [10]:
df['taken_time'] = df['end_time'] - df['start_time']
# Convert the duration to a string format (e.g., "HH:MM:SS")
df['taken_time'] = df['taken_time'].apply(lambda x: str(x).split()[2])
df

C:\Users\wingn\AppData\Local\Temp\ipykernel_20488\669833805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['taken_time'] = df['end_time'] - df['start_time']
C:\Users\wingn\AppData\Local\Temp\ipykernel_20488\669833805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['taken_time'] = df['taken_time'].apply(lambda x: str(x).split()[2])


,week_day,order_created_at_time,restaurant_id_h,order_id_h,noised_restaurant_latitude,noised_restaurant_longitude,noised_drop_latitude,noised_drop_longitude,resto_avg_driver_arrive_resto_time,resto_avg_merchant_prepare_time,resto_avg_delivery_time,no_items,restaurant_category_id,actual_time_arrival,distance,start_time,end_time,taken_time
0,Wednesday,0:03:01,f5882bd952cdb9e416a2c951d7f16935a755d9bf19fb5b...,92e4edf2207b9fd60fb9972a1bcd62d34d4b8e654f0963...,10.779663,106.604650,10.751649,106.609754,304.428571,520.857143,652.428571,2,15,0:28:13,3.164522,1900-01-01 00:03:01,1900-01-01 00:28:13,00:25:12
1,Wednesday,0:03:11,96abafeb1e03c584e9e66ddae730798c4ab63d8bcb8191...,6a2338d70aa87d482e5057e7ccc0646c5e21f12793e863...,10.798146,106.643499,10.808109,106.632191,254.941177,280.000000,420.941176,2,4,0:17:01,1.659177,1900-01-01 00:03:11,1900-01-01 00:17:01,00:13:50
2,Wednesday,0:04:12,c36562c53838cb8ed23e9de694b67c8f42ebd246ce5073...,8e700cfddfaf6b79989d7ddcdc91f10a739771a444b0a8...,10.796208,106.695271,10.791213,106.686658,220.390244,474.097561,497.658537,1,9,0:16:55,1.092479,1900-01-01 00:04:12,1900-01-01 00:16:55,00:12:43
3,Wednesday,0:04:20,d096e644f8596611ff65c5b07aed9f9d17c2ec3158edc6...,70319476604a22c258c394d9b8c879d6d32d511bd2b402...,10.753824,106.683894,10.760239,106.668003,201.444444,278.907407,583.148148,1,12,0:17:42,1.876725,1900-01-01 00:04:20,1900-01-01 00:17:42,00:13:22
4,Wednesday,0:06:57,3f8de1592f374f220e4be4456c7c0ec226729af98e99ec...,77e06752127e561d37876418b65159ec9aaa1ad4f86453...,10.796606,106.704031,10.789168,106.714135,213.000000,549.285714,674.142857,1,17,0:28:33,1.379224,1900-01-01 00:06:57,1900-01-01 00:28:33,00:21:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111531,Wednesday,16:04:30,6f3d425fbde3fd30642ca35bfa1087a53df6dbec2cf0a8...,483582428266a4c38570300a762e31541b591f71ae6042...,10.793507,106.688458,10.796297,106.698212,235.825688,518.348624,540.118182,0,12,16:19:21,1.109618,1900-01-01 16:04:30,1900-01-01 16:19:21,00:14:51
111532,Wednesday,16:05:15,3e0cbdb59cf8a8edaf262fa5147f25e3bad06cd1eea3f6...,cdb634a900e189e22735f28a92a7d2e630c63bf20eaec6...,10.790822,106.641225,10.806051,106.631188,206.848485,195.666667,554.969697,0,12,16:18:23,2.017285,1900-01-01 16:05:15,1900-01-01 16:18:23,00:13:08
111533,Wednesday,16:05:33,0a3455a92d990abf8b62f5a82cdb22d31dce076df6c00c...,60e23c3487a4778c25fbdcda8b729a966305138130d7ac...,10.755692,106.705967,10.753794,106.711555,278.488889,209.911111,486.844444,0,13,16:12:46,0.645834,1900-01-01 16:05:33,1900-01-01 16:12:46,00:07:13
111534,Wednesday,16:05:38,26b78cac2702b82b75d6b5bcead72ba2f6da3f5631ff39...,59e03cdc94a6d54698b8efd9049e9729f1d94021847c13...,10.800634,106.650167,10.797736,106.644688,176.421053,267.175439,563.964912,0,13,16:14:04,0.679662,1900-01-01 16:05:38,1900-01-01 16:14:04,00:08:26


In [11]:
df['taken_time_min']= ((df['end_time'] - df['start_time']).dt.total_seconds())//60
df['taken_time_second']= ((df['end_time'] - df['start_time']).dt.total_seconds())
df

C:\Users\wingn\AppData\Local\Temp\ipykernel_20488\2291512185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['taken_time_min']= ((df['end_time'] - df['start_time']).dt.total_seconds())//60
C:\Users\wingn\AppData\Local\Temp\ipykernel_20488\2291512185.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['taken_time_second']= ((df['end_time'] - df['start_time']).dt.total_seconds())


,week_day,order_created_at_time,restaurant_id_h,order_id_h,noised_restaurant_latitude,noised_restaurant_longitude,noised_drop_latitude,noised_drop_longitude,resto_avg_driver_arrive_resto_time,resto_avg_merchant_prepare_time,resto_avg_delivery_time,no_items,restaurant_category_id,actual_time_arrival,distance,start_time,end_time,taken_time,taken_time_min,taken_time_second
0,Wednesday,0:03:01,f5882bd952cdb9e416a2c951d7f16935a755d9bf19fb5b...,92e4edf2207b9fd60fb9972a1bcd62d34d4b8e654f0963...,10.779663,106.604650,10.751649,106.609754,304.428571,520.857143,652.428571,2,15,0:28:13,3.164522,1900-01-01 00:03:01,1900-01-01 00:28:13,00:25:12,25.0,1512.0
1,Wednesday,0:03:11,96abafeb1e03c584e9e66ddae730798c4ab63d8bcb8191...,6a2338d70aa87d482e5057e7ccc0646c5e21f12793e863...,10.798146,106.643499,10.808109,106.632191,254.941177,280.000000,420.941176,2,4,0:17:01,1.659177,1900-01-01 00:03:11,1900-01-01 00:17:01,00:13:50,13.0,830.0
2,Wednesday,0:04:12,c36562c53838cb8ed23e9de694b67c8f42ebd246ce5073...,8e700cfddfaf6b79989d7ddcdc91f10a739771a444b0a8...,10.796208,106.695271,10.791213,106.686658,220.390244,474.097561,497.658537,1,9,0:16:55,1.092479,1900-01-01 00:04:12,1900-01-01 00:16:55,00:12:43,12.0,763.0
3,Wednesday,0:04:20,d096e644f8596611ff65c5b07aed9f9d17c2ec3158edc6...,70319476604a22c258c394d9b8c879d6d32d511bd2b402...,10.753824,106.683894,10.760239,106.668003,201.444444,278.907407,583.148148,1,12,0:17:42,1.876725,1900-01-01 00:04:20,1900-01-01 00:17:42,00:13:22,13.0,802.0
4,Wednesday,0:06:57,3f8de1592f374f220e4be4456c7c0ec226729af98e99ec...,77e06752127e561d37876418b65159ec9aaa1ad4f86453...,10.796606,106.704031,10.789168,106.714135,213.000000,549.285714,674.142857,1,17,0:28:33,1.379224,1900-01-01 00:06:57,1900-01-01 00:28:33,00:21:36,21.0,1296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111531,Wednesday,16:04:30,6f3d425fbde3fd30642ca35bfa1087a53df6dbec2cf0a8...,483582428266a4c38570300a762e31541b591f71ae6042...,10.793507,106.688458,10.796297,106.698212,235.825688,518.348624,540.118182,0,12,16:19:21,1.109618,1900-01-01 16:04:30,1900-01-01 16:19:21,00:14:51,14.0,891.0
111532,Wednesday,16:05:15,3e0cbdb59cf8a8edaf262fa5147f25e3bad06cd1eea3f6...,cdb634a900e189e22735f28a92a7d2e630c63bf20eaec6...,10.790822,106.641225,10.806051,106.631188,206.848485,195.666667,554.969697,0,12,16:18:23,2.017285,1900-01-01 16:05:15,1900-01-01 16:18:23,00:13:08,13.0,788.0
111533,Wednesday,16:05:33,0a3455a92d990abf8b62f5a82cdb22d31dce076df6c00c...,60e23c3487a4778c25fbdcda8b729a966305138130d7ac...,10.755692,106.705967,10.753794,106.711555,278.488889,209.911111,486.844444,0,13,16:12:46,0.645834,1900-01-01 16:05:33,1900-01-01 16:12:46,00:07:13,7.0,433.0
111534,Wednesday,16:05:38,26b78cac2702b82b75d6b5bcead72ba2f6da3f5631ff39...,59e03cdc94a6d54698b8efd9049e9729f1d94021847c13...,10.800634,106.650167,10.797736,106.644688,176.421053,267.175439,563.964912,0,13,16:14:04,0.679662,1900-01-01 16:05:38,1900-01-01 16:14:04,00:08:26,8.0,506.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111291 entries, 0 to 111535
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   week_day                            111291 non-null  object        
 1   order_created_at_time               111291 non-null  object        
 2   restaurant_id_h                     111291 non-null  object        
 3   order_id_h                          111291 non-null  object        
 4   noised_restaurant_latitude          111291 non-null  float64       
 5   noised_restaurant_longitude         111291 non-null  float64       
 6   noised_drop_latitude                111291 non-null  float64       
 7   noised_drop_longitude               111291 non-null  float64       
 8   resto_avg_driver_arrive_resto_time  111291 non-null  float64       
 9   resto_avg_merchant_prepare_time     111291 non-null  float64       
 10  resto_avg_del

In [13]:
df = pd.read_excel(R'F:\thanh\df_1.xlsx')
df

,Unnamed: 0,week_day,order_created_at_time,restaurant_id_h,order_id_h,noised_restaurant_latitude,noised_restaurant_longitude,noised_drop_latitude,noised_drop_longitude,resto_avg_driver_arrive_resto_time,...,actual_time_arrival,distance,start_time,end_time,taken_time,taken_time_min,taken_time_second,range_time_waiting,Week_day_no,type_week_day
0,0,Wednesday,0:03:01,f5882bd952cdb9e416a2c951d7f16935a755d9bf19fb5b...,92e4edf2207b9fd60fb9972a1bcd62d34d4b8e654f0963...,10.779663,106.604650,10.751649,106.609754,304.428571,...,0:28:13,3.164522,1900-01-01 00:03:01,1900-01-01 00:28:13,00:25:12,25,1512,<=30,4,inweek
1,1,Wednesday,0:03:11,96abafeb1e03c584e9e66ddae730798c4ab63d8bcb8191...,6a2338d70aa87d482e5057e7ccc0646c5e21f12793e863...,10.798146,106.643499,10.808109,106.632191,254.941177,...,0:17:01,1.659177,1900-01-01 00:03:11,1900-01-01 00:17:01,00:13:50,13,830,<=15,4,inweek
2,2,Wednesday,0:04:12,c36562c53838cb8ed23e9de694b67c8f42ebd246ce5073...,8e700cfddfaf6b79989d7ddcdc91f10a739771a444b0a8...,10.796208,106.695271,10.791213,106.686658,220.390244,...,0:16:55,1.092479,1900-01-01 00:04:12,1900-01-01 00:16:55,00:12:43,12,763,<=15,4,inweek
3,3,Wednesday,0:04:20,d096e644f8596611ff65c5b07aed9f9d17c2ec3158edc6...,70319476604a22c258c394d9b8c879d6d32d511bd2b402...,10.753824,106.683894,10.760239,106.668003,201.444444,...,0:17:42,1.876725,1900-01-01 00:04:20,1900-01-01 00:17:42,00:13:22,13,802,<=15,4,inweek
4,4,Wednesday,0:06:57,3f8de1592f374f220e4be4456c7c0ec226729af98e99ec...,77e06752127e561d37876418b65159ec9aaa1ad4f86453...,10.796606,106.704031,10.789168,106.714135,213.000000,...,0:28:33,1.379224,1900-01-01 00:06:57,1900-01-01 00:28:33,00:21:36,21,1296,<=30,4,inweek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111286,111531,Wednesday,16:04:30,6f3d425fbde3fd30642ca35bfa1087a53df6dbec2cf0a8...,483582428266a4c38570300a762e31541b591f71ae6042...,10.793507,106.688458,10.796297,106.698212,235.825688,...,16:19:21,1.109618,1900-01-01 16:04:30,1900-01-01 16:19:21,00:14:51,14,891,<=15,4,inweek
111287,111532,Wednesday,16:05:15,3e0cbdb59cf8a8edaf262fa5147f25e3bad06cd1eea3f6...,cdb634a900e189e22735f28a92a7d2e630c63bf20eaec6...,10.790822,106.641225,10.806051,106.631188,206.848485,...,16:18:23,2.017285,1900-01-01 16:05:15,1900-01-01 16:18:23,00:13:08,13,788,<=15,4,inweek
111288,111533,Wednesday,16:05:33,0a3455a92d990abf8b62f5a82cdb22d31dce076df6c00c...,60e23c3487a4778c25fbdcda8b729a966305138130d7ac...,10.755692,106.705967,10.753794,106.711555,278.488889,...,16:12:46,0.645834,1900-01-01 16:05:33,1900-01-01 16:12:46,00:07:13,7,433,<=15,4,inweek
111289,111534,Wednesday,16:05:38,26b78cac2702b82b75d6b5bcead72ba2f6da3f5631ff39...,59e03cdc94a6d54698b8efd9049e9729f1d94021847c13...,10.800634,106.650167,10.797736,106.644688,176.421053,...,16:14:04,0.679662,1900-01-01 16:05:38,1900-01-01 16:14:04,00:08:26,8,506,<=15,4,inweek


In [14]:
df['avg_time_taken']= ((df['resto_avg_driver_arrive_resto_time'] +df['resto_avg_merchant_prepare_time']+ df['resto_avg_delivery_time']))
df=df[['week_day','order_created_at_time','order_id_h','distance','taken_time','taken_time_min','taken_time_second','range_time_waiting','Week_day_no','type_week_day','avg_time_taken']]
df

,week_day,order_created_at_time,order_id_h,distance,taken_time,taken_time_min,taken_time_second,range_time_waiting,Week_day_no,type_week_day,avg_time_taken
0,Wednesday,0:03:01,92e4edf2207b9fd60fb9972a1bcd62d34d4b8e654f0963...,3.164522,00:25:12,25,1512,<=30,4,inweek,1477.714286
1,Wednesday,0:03:11,6a2338d70aa87d482e5057e7ccc0646c5e21f12793e863...,1.659177,00:13:50,13,830,<=15,4,inweek,955.882353
2,Wednesday,0:04:12,8e700cfddfaf6b79989d7ddcdc91f10a739771a444b0a8...,1.092479,00:12:43,12,763,<=15,4,inweek,1192.146342
3,Wednesday,0:04:20,70319476604a22c258c394d9b8c879d6d32d511bd2b402...,1.876725,00:13:22,13,802,<=15,4,inweek,1063.500000
4,Wednesday,0:06:57,77e06752127e561d37876418b65159ec9aaa1ad4f86453...,1.379224,00:21:36,21,1296,<=30,4,inweek,1436.428571
...,...,...,...,...,...,...,...,...,...,...,...
111286,Wednesday,16:04:30,483582428266a4c38570300a762e31541b591f71ae6042...,1.109618,00:14:51,14,891,<=15,4,inweek,1294.292494
111287,Wednesday,16:05:15,cdb634a900e189e22735f28a92a7d2e630c63bf20eaec6...,2.017285,00:13:08,13,788,<=15,4,inweek,957.484848
111288,Wednesday,16:05:33,60e23c3487a4778c25fbdcda8b729a966305138130d7ac...,0.645834,00:07:13,7,433,<=15,4,inweek,975.244444
111289,Wednesday,16:05:38,59e03cdc94a6d54698b8efd9049e9729f1d94021847c13...,0.679662,00:08:26,8,506,<=15,4,inweek,1007.561404


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111291 entries, 0 to 111290
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   week_day               111291 non-null  object 
 1   order_created_at_time  111291 non-null  object 
 2   order_id_h             111291 non-null  object 
 3   distance               111291 non-null  float64
 4   taken_time             111291 non-null  object 
 5   taken_time_min         111291 non-null  int64  
 6   taken_time_second      111291 non-null  int64  
 7   range_time_waiting     111291 non-null  object 
 8   Week_day_no            111291 non-null  int64  
 9   type_week_day          111291 non-null  object 
 10  avg_time_taken         111291 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 9.3+ MB


## Estimated time of arrival of Food Delivery model

In [29]:
#splitting data
from sklearn.model_selection import train_test_split
x = np.array(df[["resto_avg_driver_arrive_resto_time", 
                   "resto_avg_merchant_prepare_time", 
                   "resto_avg_delivery_time",
                   "distance"]])
y = np.array(df[["taken_time_second"]])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.10, 
                                                random_state=42)

# creating the LSTM neural network model
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# training the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=9)


Epoch 1/9

100161/100161 [==============================] - 476s 5ms/step - loss: 833395.4375
Epoch 2/9
100161/100161 [==============================] - 483s 5ms/step - loss: 786922.5625
Epoch 3/9
100161/100161 [==============================] - 463s 5ms/step - loss: 766083.6250
Epoch 4/9
100161/100161 [==============================] - 473s 5ms/step - loss: 755163.9375
Epoch 5/9
100161/100161 [==============================] - 462s 5ms/step - loss: 746832.4375
Epoch 6/9
100161/100161 [==============================] - 759s 8ms/step - loss: 735622.5625
Epoch 7/9
100161/100161 [==============================] - 488s 5ms/step - loss: 750378.1250
Epoch 8/9
100161/100161 [==============================] - 489s 5ms/step - loss: 738794.7500
Epoch 9/9
100161/100161 [==============================] - 454s 5ms/step - loss: 736607.1875


Now let’s test the performance of our model by giving inputs to predict the food delivery time:

In [31]:
model.save('my_trained_model.h5')

C:\Users\wingn\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [32]:
print("Food Delivery Time Prediction")
a = int(input("Time for drivers to travel from the accept location to the restaurant: "))
b = float(input("Time Restaurant preparation: "))
c = float(input("Time the driver takes from the restaurant to the delivery address: "))
d = int(input("Total Distance: "))

features = np.array([[a, b, c,d]])
print("Predicted Delivery Time in seconds = ", model.predict(features))

Food Delivery Time Prediction
Time for drivers to travel from the accept location to the restaurant: 23
Time Restaurant preparation: 509
Time the driver takes from the restaurant to the delivery address: 627
Total Distance: 4
1/1 [==============================] - 2s 2s/step
Predicted Delivery Time in seconds =  [[1710.3436]]
